

---
# HuggingFace :) Stable Diffusion model with backend support from FastAPI on Google Colab 

---
Implemented a very simple UI using HTML .


**Create account on ngrok to get your authtoken 
ngrok provides tunneling so that the API can be exposed to the public domain .**


In [1]:
!pip install fastapi nest-asyncio pyngrok uvicorn colabcode
!pip install git+https://github.com/ml-research/safe-latent-diffusion &> /dev/null
!pip install ftfy &> /dev/null
!curl -LJO https://github.com/crowsonkb/simulacra-aesthetic-models/blob/master/models/sac_public_2022_06_29_vit_b_16_linear.pth?raw=true
!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio==0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!git clone https://github.com/openai/CLIP.git
!cd CLIP
!pip install .
!cd ..


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.6 MB/s 
     |████████████████████████████████| 761 kB 9.0 MB/s 
     |████████████████████████████████| 56 kB 4.8 MB/s 
     |████████████████████████████████| 64 kB 3.4 MB/s 
     |████████████████████████████████| 80 kB 10.4 MB/s 
     |████████████████████████████████| 58 kB 6.7 MB/s 
     |████████████████████████████████| 8.3 MB 60.2 MB/s 
     |████████████████████████████████| 45 kB 3.8 MB/s 
     |████████████████████████████████| 9.8 MB 69.6 MB/s 
     |████████████████████████████████| 346 kB 77.6 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 423 kB 80.7 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 
     |████████████████████████████████| 273 kB 76.3 MB/s 
     |████████████████████████████████| 133 kB 82.3 MB/s 
     |████████████████████████████████| 62 kB 1

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3111  100  3111    0     0   9971      0 --:--:-- --:--:-- --:--:--  9971
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 1.4 MB/s eta 0:11:44tcmalloc: large alloc 1147494400 bytes == 0x65fac000 @  0x7efe867f0615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x4fd8b5 0x4997c7 0x4fd8b5 0x49abe4 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x5d8868 0x5da092 0x587116 0x5d8d8c 0x55dc1e 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4990ca 0x5d8868 0x4997a2 0x4fd8b5 0x49abe4
     |███████████████

Login for ngrok and HuggingFace

In [2]:
from pyngrok import ngrok 

!ngrok authtoken <your ngrok authtoken >

ngrok.kill()

# !huggingface-cli login

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
!killall ngrok

ngrok: no process found


HTML 

In [10]:
%%writefile index.html
<html>
    <head><h1>Stable diffusion model</h1><br><br></head>
    <body>
    <div> 
      <form method = "post" id ="myForm"> 
        <label for = "prompt">Prompt</label> <br>
        <textarea id="prompt" name="prompt" rows="4" cols="50">
          Enter String 
        </textarea>
        <br><br>
        <label for = "numbers" >Enter count of images</label> <br> 
        <input type = "number" id="numbers" name="N" min="1"><br> <br>
        <input type="button" value="Start" onclick="submitForm()">
      </form>
    <div id = "big"> 
    </div> 
    <script>
         function submitForm() {
             var formElement = document.getElementById('myForm');
             var data = new FormData(formElement);
             var list = document.getElementById("big") 
             while (list.hasChildNodes()) {
                list.removeChild(list.firstChild);
              }
             var loading = document.createElement("h4")
             loading.id = "loading"
             loading.innerHTML = "Generating images..... Please wait "
             document.getElementById("big").appendChild(loading)
             fetch('calculate', {
                   method: 'POST',
                   headers: {
                     'Content-Type': 'application/json'
                    },
                   body: JSON.stringify(Object.fromEntries(data))
                 })
                 .then(resp => resp.json())  // or, resp.json(), etc.
                 .then(data => {
                   data = JSON.parse(data);

                   document.getElementById("loading").remove()

                   var responses = document.createElement("div") ; 
                   responses.id = "responses" 
                   document.getElementById("big").appendChild(responses)

                   var i =0 


                   for(; i < data.length;i++)
                   {
                     var div = document.createElement("div");
                     div.id = "div" + i.toString() 
                     document.getElementById("responses").appendChild(div) 
                     
                     var image = document.createElement("img") 
                     image.src = data[i]['image'] 
                     document.getElementById(div.id).appendChild(image)

                     var score = document.createElement("p") ; 
                     score.innerHTML = data[i]['score'] 
                     document.getElementById(div.id).appendChild(score)
                   }
                   var para = document.createElement("h3") 
                   para.innerHTML = "Best image\n"
                   document.getElementById("big").appendChild(para) 

                   var div = document.createElement("div");
                   div.id = "div" +  i.toString() ;
                   document.getElementById("big").appendChild(div) 
                     
                   var image = document.createElement("img") 
                   image.src = data[i-1]['image']
                   document.getElementById(div.id).appendChild(image)

                   var score = document.createElement("p") ; 
                   score.innerHTML = data[i-1]['score'] 
                   document.getElementById(div.id).appendChild(score)
                   
                   
                 })
                 .catch(error => {
                   console.error(error);
                 });
         }
      </script>
    </body>
  </html> 

Overwriting index.html


Model 

In [5]:
from sld import SLDPipeline
import torch
stable_diffusion_src_path = "CompVis/stable-diffusion-v1-4"
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

with torch.no_grad() : 
  pipe = SLDPipeline.from_pretrained(
      stable_diffusion_src_path,
      #safety_checker = None,
  )
path= 'model.pth'
torch.save(pipe  , path )

Downloading:   0%|          | 0.00/543 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/342 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/743 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/522 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

In [6]:
import torch
import torch.nn as nn 
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms import functional as TF
from CLIP import clip
from tqdm import tqdm
from PIL import Image

path = 'model.pth'

class AestheticMeanPredictionLinearModel(nn.Module):
    def __init__(self, feats_in):
        super().__init__()
        self.linear = nn.Linear(feats_in, 1)

    def forward(self, input):
        x = F.normalize(input, dim=-1) * input.shape[-1] ** 0.5
        return self.linear(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

clip_model_name = 'ViT-B/16'
clip_model = clip.load(clip_model_name, jit=False, device=device)[0]
clip_model.eval().requires_grad_(False)

asthetic_model = AestheticMeanPredictionLinearModel(512)
asthetic_model.load_state_dict(
    torch.load("/content/sac_public_2022_06_29_vit_b_16_linear.pth?raw=true")
)
asthetic_model = asthetic_model.to(device)
gen_model = torch.load(path , map_location = device)

100%|███████████████████████████████████████| 335M/335M [00:11<00:00, 30.5MiB/s]


FastAPI
 

In [7]:

async def getImages(model , text , count) : 
  
  if model : 
    gen = torch.Generator(device=device)
    gen.manual_seed(3503638049)
    print(".................Generating images ...............")
    with torch.no_grad() : 
      out = model(prompt= text, generator=gen, guidance_scale=1 , num_images_per_prompt =  count ) 
    
    return out[0]

async def scoreImages(asthetic_model , clip_model   , images )  : 
    print("............Scoring images.................")
    normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                  std=[0.26862954, 0.26130258, 0.27577711])
    
    scores = []
  
    for image in tqdm(images):
        img = image.convert('RGB')
        img = TF.resize(img, 224, transforms.InterpolationMode.LANCZOS)
        img = TF.center_crop(img, (224,224))
        img = TF.to_tensor(img).to(device)
        img = normalize(img)
        clip_image_embed = F.normalize(
            clip_model.encode_image(img[None, ...]).float(),
            dim=-1)
        score = asthetic_model(clip_image_embed)
        scores.append((score.item(),image))
    
    
    scores.sort(key=lambda x: x[0])
     
    return  scores 


In [8]:
import torch 
from fastapi import FastAPI, Form , Request , Response 
import uvicorn
from colabcode import ColabCode
from fastapi.templating import Jinja2Templates
from fastapi.responses import HTMLResponse
from fastapi.middleware.cors import CORSMiddleware
import io
import asyncio 
import json 
import base64

templates = Jinja2Templates(directory = "/content/")

app = FastAPI() 

origins = [
    "http://localhost",
    "http://localhost:8000"
    "http://localhost:8080",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

TIME_OUT_ERROR = 20 

@app.post("/calculate") 
async def calculate(request : Request ) : 

  data = await request.json() 
  print(data)
  print("...................... inside the  model ............ ")
  
  prompt = data['prompt'] 
  cnt = int(data['N'] )

  
  img = await getImages(gen_model , prompt , cnt )
  scores = await scoreImages(asthetic_model  , clip_model , img )
  print(".................The image output is this .......................")
  
  img_arr = [] 

  l = []
  for i in range(cnt) : 

    img_byte_arr = io.BytesIO()
    scores[i][1].save(img_byte_arr, format='PNG')
    img_byte = base64.b64encode(img_byte_arr.getvalue()) 
    img_byte = "data:image/png;base64," + img_byte.decode('utf-8')
    obj = {"image": img_byte , "score" : scores[i][0]}
    l.append(obj)

  return json.dumps(l)   

@app.get("/" , response_class = HTMLResponse)
def home(request : Request ) : 

  return templates.TemplateResponse("index.html" , {"request": request })


In [11]:
server = ColabCode(port = 8000, code=False )
server.run_app(app=app)

Public URL: NgrokTunnel: "https://10b8-34-139-180-71.ngrok.io" -> "http://localhost:8000"


INFO:     Started server process [212]
INFO:uvicorn.error:Started server process [212]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     103.205.115.86:0 - "GET / HTTP/1.1" 200 OK
INFO:     103.205.115.86:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
{'prompt': 'man', 'N': '3'}
...................... inside the  model ............ 
.................Generating images ...............


  0%|          | 0/51 [00:00<?, ?it/s]

............Scoring images.................


100%|██████████| 3/3 [00:00<00:00, 53.93it/s]

.................The image output is this .......................


INFO:     103.205.115.86:0 - "POST /calculate HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [212]
INFO:uvicorn.error:Finished server process [212]


In [12]:
!killall ngrok 

ngrok: no process found
